<a href="https://colab.research.google.com/github/philqc/Structured_Prediction/blob/master/DVN_Img_Seg_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'
!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import datasets, transforms, utils
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler
import random
import os
import pickle
import time
import math
import copy
import pdb
from skimage import io

    100% |████████████████████████████████| 753.6MB 59.0MB/s 
    100% |████████████████████████████████| 2.0MB 12.1MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.


For some reason, we are getting AttributeError: module 'PIL.Image' has no attribute 'register_extensions if we don't install pillow version 4.1.1

In [0]:
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

    100% |████████████████████████████████| 5.7MB 8.0MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1


Load the dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

# directory path to store results/plots/models
dir_path = 'drive/My Drive/projet_asp/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


Auxiliary functions

In [0]:
class Adam:
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))

        self.v = torch.zeros_like(params)
        self.m = torch.zeros_like(params)
        self.betas = betas
        self.lr = lr
        self.eps = eps
        self.t = 0

    def update(self, gradients):
        # update time step
        self.t += 1

        self.m = self.betas[0] * self.m + (1 - self.betas[0]) * gradients
        self.v = self.betas[1] * self.v + (1 - self.betas[1]) * (gradients ** 2)

        # Bias corrected first and second moment estimates
        mean = self.m / (1 - self.betas[0] ** self.t)
        variance = self.v / (1 - self.betas[1] ** self.t)

        update = self.lr * mean / (torch.sqrt(variance) + self.eps)
        return update


class SGD:

    def __init__(self, params, lr=0.5, momentum=0., weight_decay=0.0):
        """
        SGD with momentum for the inference part of
        training/testing. This speeds up training by using
        autograd.grad of the loss with respect to only the inputs
        but optimizer of pytorch is not compatible, so we make
        our own optimizer function
        """
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if momentum < 0.0:
            raise ValueError("Invalid momentum value: {}".format(momentum))
        if weight_decay < 0.0:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))

        self.lr = lr
        self.momentum = momentum
        self.v = torch.zeros_like(params)

    def update(self, gradients):
        self.v = self.momentum * self.v + self.lr * gradients
        return self.v


class MyDataset(Dataset):

    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

    def __len__(self):
        return len(self.inputs)


def print_a_sentence(x, y, txt_inputs, txt_labels):
    """ To visualize the data """
    print('-----------------')
    for i, x in enumerate(x):
        if x == 1:
            print(txt_inputs[i])
    
    print('-----------------')
    print('TAGS:')
    for i, y in enumerate(y):
        if y == 1:
            print(txt_labels[i])
    

def compute_f1_score(labels, outputs):
    """ 
    Compute the example averaged (macro average) F1 measure
    """
    assert labels.shape == outputs.shape

    f1 = []
    for i in range(len(outputs)):
        f1.append(f1_score(labels[i], outputs[i]))

    return np.mean(f1)


def plot_results(results):
    """
    Parameters:
    ----------
    results: dictionary with the train/valid loss
    and the f1 scores
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    ax1.set_title('Validation Loss')
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epochs')
    ax2.set_title('Validation F1 Score')
    ax2.set_ylabel('F1 Score')
    ax2.set_xlabel('epochs')

    ax1.plot(results['loss_train'], label='loss_train')
    ax1.plot(results['loss_valid'], label='loss_valid')
    ax2.plot(results['f1_valid'])

    ax1.legend()
    plt.show()


def plot_aggregate_results(results_path, add_title=''):

    array_results = []
    for filename in os.listdir(results_path):
        if filename.endswith('.pkl'):
            with open(os.path.join(results_path, filename), 'rb') as fin:
                array_results.append(pickle.load(fin))

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
    ax1.set_title('Validation Loss ' + add_title)
    ax1.set_ylabel('loss')
    ax1.set_xlabel('epochs')
    ax2.set_title('Validation F1 Score ' + add_title)
    ax2.set_ylabel('F1 Score')
    ax2.set_xlabel('epochs')

    # max number of epochs
    max_ep = 60

    for res in array_results:

        if res['name'] == 'SPEN_bibtex':
            res['name'] = 'SPEN'

        label = res['name']

        # ax1.plot(res['loss_train'][:max_ep], label=label)
        if res['name'] != 'SPEN':
            ax1.plot(res['loss_valid'][:max_ep], label=label)
        ax2.plot(res['f1_valid'][:max_ep], label=label)

    ax1.legend()
    ax2.legend()
    plt.show()



Deep Value Network

In [0]:
# build the dataset, generate the "training tuple"
class WeizmannHorseDataset(Dataset):
    """ Weizmann Horse Dataset """
    
    def __init__(self, img_dir, mask_dir, test_set, transform=None):
        """
        Args:
            img_dir(string): Path to the image file (training image)
            mask_dir(string): Path to the mask file (segmentation result)
            test_set(bool): if we want test set or train set
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.img_dir = img_dir
        self.mask_dir = mask_dir

        all_img_names = os.listdir(img_dir)
        all_mask_names = os.listdir(mask_dir)

        self.img_names = []
        self.mask_names = []
        for i, name in enumerate(all_img_names):
            img_number = ''.join([n for n in name if n.isdigit()])
            if int(img_number) >= 200:
                if test_set:
                    self.img_names.append(name)
                    self.mask_names.append(all_mask_names[i])
            else:
                self.img_names.append(name)
                self.mask_names.append(all_mask_names[i])

        assert len(self.mask_names) == len(self.img_names)
        self.transform = transform
        
    def __len__(self):
        return len(self.img_names)
        
    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.img_names[idx])
        mask_name = os.path.join(self.mask_dir, self.mask_names[idx])
        
        image = io.imread(img_name)
        mask = io.imread(mask_name)
        
        if self.transform:
            image = self.transform(image)
            
            #create a channel for mask so as to transform
            mask = self.transform(np.expand_dims(mask, axis=2))
            
        return image, mask


class ConvNet(nn.Module):

    # define each layer of neural network
    def __init__(self, non_linearity=nn.ELU()):
        super().__init__()
        # Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        self.conv1 = nn.Conv2d(4, 64, 5, 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 5, 2)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, 5, 2)
        self.bn3 = nn.BatchNorm2d(128)
        # Linear(in_features, out_features, bias=True)
        self.fc1 = nn.Linear(128 * 4 * 4, 384)

        self.fc2 = nn.Linear(384, 192)
        self.fc3 = nn.Linear(192, 1)
        self.non_linearity = non_linearity

    # define how input will be processed through those layers
    def forward(self, x, y):
        # We first concatenate the img and the mask
        z = torch.cat((x, y), 1)
        z = self.non_linearity(self.bn1(self.conv1(z)))
        z = self.non_linearity(self.bn2(self.conv2(z)))
        z = self.non_linearity(self.bn3(self.conv3(z)))
        # don't forget to flatten before connect to FC layers
        z = z.view(-1, 128 * 4 * 4)
        z = self.non_linearity(self.fc1(z))
        # apply dropout on the first FC layer as paper mentioned
        z = F.dropout(z, p=0.75)
        z = self.non_linearity(self.fc2(z))
        z = self.non_linearity(self.fc3(z))
        return z


class DeepValueNetwork:

    def __init__(self, dataset, use_cuda, add_adversarial=True,
                 add_ground_truth=False, stratified_sampling=False, batch_size=16, batch_size_eval=16,
                 learning_rate=1e-3, inf_lr=0.5, feature_dim=(32, 32), label_dim=(32, 32),
                 non_linearity=nn.ELU()):
        """
        Parameters
        ----------
        use_cuda: boolean
            true if we are using gpu, false if using cpu
        learning_rate : float
            learning rate for updating the value network parameters
        inf_lr : float
            learning rate for the inference procedure
        add_adversarial: bool
            Generate adversarial tuples while training.
            (Usually outperforms stratified sampling and adding ground truth)
        stratified_sampling: bool (Not yet implemented)
            Sample y proportional to its exponential oracle value.
            Sample from the exponentiated value distribution using stratified sampling.
        add_ground_truth: bool
            Simply add the ground truth outputs y* with some probably p while training.
        """

        self.device = torch.device("cuda" if use_cuda else "cpu")

        self.add_adversarial = add_adversarial
        self.add_ground_truth = add_ground_truth
        self.stratified_sampling = stratified_sampling
        if self.stratified_sampling:
            raise ValueError('Stratified sampling is not yet implemented!')
        if self.add_ground_truth and self.add_adversarial:
            raise ValueError('Adversarial examples and Adding Ground Truth are both set to true !')

        self.feature_dim = feature_dim
        self.label_dim = label_dim
        self.inf_lr = inf_lr

        # Deep Value Network is just a ConvNet
        self.model = ConvNet(non_linearity).to(self.device)

        self.loss_fn = nn.BCEWithLogitsLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)

        self.batch_size = batch_size
        self.batch_size_eval = batch_size_eval

        self.n_train = int(len(dataset) * 0.80)
        self.n_valid = len(dataset) - self.n_train

        print('Using a {} train {} valid data-split'.format(self.n_train, self.n_valid))
        indices = list(range(len(dataset)))
        random.shuffle(indices)

        self.train_loader = DataLoader(
            dataset,
            batch_size=self.batch_size,
            sampler=SubsetRandomSampler(indices[:self.n_train]),
            pin_memory=use_cuda
        )

        self.valid_loader = DataLoader(
            dataset,
            batch_size=self.batch_size_eval,
            sampler=SubsetRandomSampler(indices[self.n_train:]),
            pin_memory=use_cuda
        )

        # turn on/off
        self.training = False


    def get_oracle_value(self, pred_labels, gt_labels):
        """
        Compute the ground truth value, i.e. v*(y, y*)
        of some predicted labels, where v*(y, y*)
        is the relaxed version of the F1 Score.
        """
        if pred_labels.shape != gt_labels.shape:
            raise ValueError('Invalid labels shape: gt = ', gt_labels.shape, 'pred = ', pred_labels.shape)

        if not self.training:
            # No relaxation, 0-1 only
            pred_labels = torch.where(pred_labels >= 0.5, 
                                      torch.ones(1).to(self.device), 
                                      torch.zeros(1).to(self.device))
            pred_labels = pred_labels.float()

        pred_labels = torch.flatten(pred_labels).reshape(pred_labels.size()[0], -1)
        gt_labels = torch.flatten(gt_labels).reshape(gt_labels.size()[0], -1)

        intersect = torch.sum(torch.min(pred_labels, gt_labels), dim=1)
        union = torch.sum(torch.max(pred_labels, gt_labels), dim=1)

        # for numerical stability
        epsilon = torch.full(union.size(), 10 ** -8).to(self.device)

        relaxed_iou = intersect / torch.max(epsilon, union)
        # we want a (Batch_size x 1) tensor
        relaxed_iou = relaxed_iou.view(-1, 1)
        #pdb.set_trace()
        return relaxed_iou

    def get_ini_labels(self, x, gt_labels=None):
        """
        Get the tensor of predicted labels
        that we will do inference on
        """
        y = torch.zeros(x.size()[0], 1, self.label_dim[0], self.label_dim[1],
                        dtype=torch.float32, device=self.device)

        if gt_labels is not None:
            # 50%: Start from GT; rest: start from zeros
            gt_indices = torch.rand(gt_labels.shape[0]).float().to(self.device) > 0.5
            y[gt_indices] = gt_labels[gt_indices]

        # Set requires_grad=True after in_place operation (changing the indices)
        y.requires_grad = True
        return y

    def generate_output(self, x, gt_labels):
        """
        Generate an output y to compute
        the loss v(y, y*) --> we can use different
        techniques to generate the output
        1) Gradient based inference
        2) Simply add the ground truth outputs
        2) Generating adversarial tuples
        3) TODO: Stratified Sampling: Random samples from Y, biased towards y*
        """

        if self.add_adversarial and self.training and np.random.rand() >= 0.5:
            # In training: Generate adversarial examples 50% of the time
            init_labels = self.get_ini_labels(x, gt_labels=gt_labels)
            pred_labels = self.inference(x, init_labels, gt_labels=gt_labels, num_iterations=1)
        elif self.add_ground_truth and self.training and np.random.rand() >= 0.5:
            # In training: If add_ground_truth=True, add ground truth outputs
            # to provide some positive examples to the network
            pred_labels = gt_labels
        else:
            init_labels = self.get_ini_labels(x)
            pred_labels = self.inference(x, init_labels)

        return pred_labels

    def inference(self, x, y, gt_labels=None, num_iterations=20):

        if self.training:
            self.model.eval()

        #optim_inf = SGD(y, lr=2, momentum=0.9)
        optim_inf = Adam(y, lr=0.5)

        with torch.enable_grad():

            for i in range(num_iterations):

                if gt_labels is not None:  # Adversarial
                    oracle = self.get_oracle_value(y, gt_labels)
                    output = self.model(x, y)
                    # this is the BCE loss with logits
                    value = self.loss_fn(output, oracle)
                else:
                    output = self.model(x, y)
                    value = torch.sigmoid(output)

                grad = torch.autograd.grad(value, y, grad_outputs=torch.ones_like(value),
                                           only_inputs=True)

                y_grad = grad[0].detach()
                #pdb.set_trace()
                y = y + optim_inf.update(y_grad)
                # Project back to the valid range
                y = torch.clamp(y, 0, 1)
               

        if self.training:
            self.model.train()

        return y

    def train(self, ep):

        self.model.train()
        self.training = True

        time_start = time.time()
        t_loss, t_size = 0, 0

        for batch_idx, (inputs, targets) in enumerate(self.train_loader):

            inputs, targets = inputs.to(self.device), targets.to(self.device)
            inputs, targets = inputs.float(), targets.float()

            t_size += len(inputs)

            self.model.zero_grad()

            pred_labels = self.generate_output(inputs, targets)
            oracle = self.get_oracle_value(pred_labels, targets)
            output = self.model(inputs, pred_labels)

            loss = self.loss_fn(output, oracle)
            t_loss += loss.item()

            loss.backward()
            self.optimizer.step()

            if batch_idx % 2 == 0:
                print('\rTraining Epoch {} [{} / {} ({:.0f}%)]: Time per epoch: {:.2f}s; Avg_Loss = {:.5f}; IOU = {:.2f}%'
                      ''.format(ep, t_size, self.n_train, 100 * t_size / self.n_train,
                                (self.n_train / t_size) * (time.time() - time_start), t_loss / t_size, 100*oracle.mean()),
                      end='')

        t_loss /= t_size
        self.training = False
        print('')
        return t_loss

    def valid(self, loader, test_set=False):

        self.model.eval()
        self.training = False

        loss, t_size = 0, 0
        mean_iou = []

        with torch.no_grad():
            for (inputs, targets) in loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                inputs, targets = inputs.float(), targets.float()
                t_size += len(inputs)

                pred_labels = self.generate_output(inputs, targets)
                oracle = self.get_oracle_value(pred_labels, targets)
                output = self.model(inputs, pred_labels)
                
                loss += self.loss_fn(oracle, output)

                mean_iou.append(oracle.mean())
        
        mean_iou = torch.stack(mean_iou)
        mean_iou = torch.mean(mean_iou)
        loss /= t_size

        if test_set:
            print('Test set: Avg_Loss = {:.2f};  IOU = {:.2f}%'
                  ''.format(loss.item(), 100 * mean_iou))
        else:
            print('Validation set: Avg_Loss = {:.2f}; IOU = {:.2f}%'
                  ''.format(loss.item(), 100 * mean_iou))

        return loss.item(), mean_iou


Load bibtex and choose to use cpu or gpu

In [0]:
# Use GPU if it is available
use_cuda = torch.cuda.is_available()
print('using cuda =', use_cuda)
    
image_dir = dir_path + './images'
mask_dir = dir_path + './masks'

WhorseDataset = WeizmannHorseDataset(image_dir, mask_dir, test_set=False,
                                     transform=transforms.Compose([
                                               transforms.ToPILImage(),
                                               transforms.Resize(size=(32, 32)),
                                               transforms.ToTensor()
                                     ]))


using cuda = True


In [0]:
Run the model

In [0]:


DVN = DeepValueNetwork(WhorseDataset, use_cuda,
                       add_adversarial=False, add_ground_truth=True,
                       batch_size=32, batch_size_eval=32)

# Decay the learning rate by a factor of gamma every step_size # of epochs
scheduler = torch.optim.lr_scheduler.StepLR(DVN.optimizer, step_size=30, gamma=0.1)

results = {'name': 'DVN_Whorse', 'loss_train': [],
           'loss_valid': [], 'IOU_valid': []}

save_results_file = os.path.join(dir_path, results['name'] + '.pkl')

for epoch in range(60):
  loss_train = DVN.train(epoch)
  loss_valid, IOU_valid = DVN.valid(DVN.valid_loader)
  scheduler.step()
  results['loss_train'].append(loss_train)
  results['loss_valid'].append(loss_valid)
  results['IOU_valid'].append(IOU_valid)

with open(save_results_file, 'wb') as fout:
  pickle.dump(results, fout)

plot_results(results)
torch.save(DVN.model.state_dict(), dir_path + '/' + results['name'] + '.pth')


Using a 160 train 40 valid data-split
Training Epoch 0 [160 / 160 (100%)]: Time per epoch: 1.23s; Avg_Loss = 0.02381; IOU = 100.00%
Validation set: Avg_Loss = 0.03; IOU = 42.37%
Training Epoch 1 [160 / 160 (100%)]: Time per epoch: 1.42s; Avg_Loss = 0.02092; IOU = 46.00%
Validation set: Avg_Loss = 0.02; IOU = 47.45%
Training Epoch 2 [160 / 160 (100%)]: Time per epoch: 2.00s; Avg_Loss = 0.02602; IOU = 10.50%
Validation set: Avg_Loss = 0.04; IOU = 11.50%
Training Epoch 3 [160 / 160 (100%)]: Time per epoch: 2.00s; Avg_Loss = 0.02345; IOU = 31.93%
Validation set: Avg_Loss = 0.05; IOU = 35.04%
Training Epoch 4 [160 / 160 (100%)]: Time per epoch: 1.37s; Avg_Loss = 0.02331; IOU = 100.00%
Validation set: Avg_Loss = 0.04; IOU = 43.08%
Training Epoch 5 [160 / 160 (100%)]: Time per epoch: 1.39s; Avg_Loss = 0.01946; IOU = 100.00%
Validation set: Avg_Loss = 0.01; IOU = 48.05%
Training Epoch 6 [160 / 160 (100%)]: Time per epoch: 1.98s; Avg_Loss = 0.02578; IOU = 47.97%
Validation set: Avg_Loss = 0.02;

KeyboardInterrupt: ignored